## Frequency Assignment

The design of wireless networks, such as cell phone networks, involves assigning communication frequencies to devices. These communication frequencies can be separated into channels. The geographical area covered by a network can be divided into hexagonal cells, where each cell has a base station that covers a given area. Each cell requires a different number of channels, based on usage statistics and each cell has a set of neighbor cells, based on the geographical distances. The design of an efficient mobile network involves selecting subsets of channels for each cell, avoiding interference between calls in the same cell and in neighboring cells. Also, for economical reasons, the total bandwidth in use must be minimized, i.e., the total number of different channels used. One of the first real cases discussed in literature are the Philadelphia [Ande73] instances, with the structure depicted bellow:

Each cell has a demand with the required number of channels drawn at the center of the hexagon, and a sequential id at the top left corner. Also, in this example, each cell has a set of at most 6 adjacent neighboring cells (distance 1). The largest demand (8) occurs on cell 2. This cell has the following adjacent cells, with distance 1: (1, 6). The minimum distances between channels in the same cell in this example is 3 and channels in neighbor cells should differ by at least 2 units.

A generalization of this problem (not restricted to the hexagonal topology), is the Bandwidth Multicoloring Problem (BMCP), which has the following input data:

𝑁:
set of cells, numbered from 1 to 𝑛;

𝑟𝑖∈ℤ+:
demand of cell 𝑖∈𝑁, i.e., the required number of channels;

𝑑𝑖,𝑗∈ℤ+:
minimum distance between channels assigned to nodes 𝑖 and 𝑗, 𝑑𝑖,𝑖 indicates the minimum distance between different channels allocated to the same cell.

Given an upper limit 𝑢⎯⎯⎯ on the maximum number of channels 𝑈={1,…,𝑢⎯⎯⎯} used, which can be obtained using a simple greedy heuristic, the BMPC can be formally stated as the combinatorial optimization problem of defining subsets of channels 𝐶1,…,𝐶𝑛 while minimizing the used bandwidth and avoiding interference:

$
\begin{align*}
&Minimize& &\max_{c \in C_1, \cup ,..., C_n} c \\
&s.t.& &\|c_1 - c_2\| \geqq d_{i, j} \forall (i, j) \in N × N, (c_1, c_2) \forall C_i × C_j \\
&& &C_i \subseteq U \forall i \in N \\
&& &\|C_i\|=r_i \forall i \in N \\
\end{align*}
$

This problem can be formulated as a mixed integer program with binary variables indicating the composition of the subsets: binary variables 𝑥(𝑖,𝑐) indicate if for a given cell 𝑖 channel 𝑐 is selected (𝑥(𝑖,𝑐)=1) or not (𝑥(𝑖,𝑐)=0). The BMCP can be modeled with the following MIP formulation:

$
\begin{align*}
&Minimize& &z \\
&s.t.& &\sum_{c=1}^{u} = r_i \forall i \in N \\
&& &x_{i, c} + x_{j, c'} \leqq 1 \forall (i, j, c, c') \in N × N × U × U : i ≠ j \wedge \|c - c'\|<d_{i, j} \\
&& &x_{i, c} + x_{i, c'} \leqq 1 \forall i, c \in N × U, c' \in \{c, +1 ..., min(c+d_{i,i},u)\} \\
&& &x_{i, c} \in \{0, 1\} \forall i \in N, c \in U \\
&& &z \geqq 0 \\
\end{align*}
$

Follows the example of a solver for the BMCP using the previous MIP formulation:

In [ ]:
from itertools import product
from mip import Model, xsum, minimize, BINARY

# number of channels per node
r = [3, 5, 8, 3, 6, 5, 7, 3]

# distance between channels in the same node (i, i) and in adjacent nodes
#      0  1  2  3  4  5  6  7
d = [[3, 2, 0, 0, 2, 2, 0, 0],   # 0
     [2, 3, 2, 0, 0, 2, 2, 0],   # 1
     [0, 2, 3, 0, 0, 0, 3, 0],   # 2
     [0, 0, 0, 3, 2, 0, 0, 2],   # 3
     [2, 0, 0, 2, 3, 2, 0, 0],   # 4
     [2, 2, 0, 0, 2, 3, 2, 0],   # 5
     [0, 2, 2, 0, 0, 2, 3, 0],   # 6
     [0, 0, 0, 2, 0, 0, 0, 3]]   # 7

N = range(len(r))

# in complete applications this upper bound should be obtained from a feasible
# solution produced with some heuristic
U = range(sum(d[i][j] for (i, j) in product(N, N)) + sum(el for el in r))

m = Model()

x = [[m.add_var('x({},{})'.format(i, c), var_type=BINARY)
      for c in U] for i in N]

z = m.add_var('z')
m.objective = minimize(z)

for i in N:
    m += xsum(x[i][c] for c in U) == r[i]

for i, j, c1, c2 in product(N, N, U, U):
    if i != j and c1 <= c2 < c1+d[i][j]:
        m += x[i][c1] + x[j][c2] <= 1

for i, c1, c2 in product(N, U, U):
    if c1 < c2 < c1+d[i][i]:
        m += x[i][c1] + x[i][c2] <= 1

for i, c in product(N, U):
    m += z >= (c+1)*x[i][c]

m.optimize(max_nodes=30)

if m.num_solutions:
    for i in N:
        print('Channels of node %d: %s' % (i, [c for c in U if x[i][c].x >=
                                               0.99]))

Welcome to the CBC MILP Solver 
Version: devel 
Build Date: Nov 15 2020 

Starting solution of the Linear programming relaxation problem using Dual Simplex

Coin0506I Presolve 6282 (-1199) rows, 873 (0) columns and 13420 (-2398) elements
Clp0014I Perturbing problem by 0.001% of 1.797539 - largest nonzero change 0.00011682946 ( 0.0064994119%) - largest zero change 0.00011653858
Clp0000I Optimal - objective value 1.8896391
Coin0511I After Postsolve, objective 1.8896391, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 1.889639058 - 367 iterations time 0.022, Presolve 0.01

Starting MIP optimization
Cgl0003I 0 fixed, 0 tightened bounds, 4943 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 4582 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 3545 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1784 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 712 strengthened rows, 0 